In [60]:
import pandas as pd
from postgres_connect import *
import numpy as np

In [81]:
#change below for date----------------------------------------------------------------
query = '''
        select 
    r.awb
,   o.shipping_city
,   o.shipping_pincode::integer
,   o.sort_code
from revenue_24_jun as r
left join ops_main as o on o.awb = r.awb
where r.attributed_month::date >= '2024-02-01'
and r.attributed_month::date <= '2024-05-30'       
'''
df_revenue = get_df_from_sql('Yes',query,'tunnel-ssh.cer')
df_revenue.head()

,awb,shipping_city,shipping_pincode,sort_code
0,GS1007447048,Delhi,110075.0,DEL/DWRK/DK_30
1,GS1472650591,Delhi,110066.0,DEL/HKMN/SD_89
2,GS2064847292,Delhi,110049.0,DEL/KTWS/SD_84
3,GS1493310339,Delhi,110025.0,DEL/HKMN/SV_88
4,GS1898033811,Delhi,110085.0,DEL/RHNP/AP_19


In [82]:
df_revenue[df_revenue['awb']=='GS2146753063']

,awb,shipping_city,shipping_pincode,sort_code
1874813,GS2146753063,NCR,121009.0,NCR / FRBD / 70


In [96]:
df_march = pd.read_csv('margin_april.csv')
df_march.head()

,Unnamed: 0,awb,warehouse_city,drop_pincode,shipment_status,mop,rpo,shipping_city,num_rider_id,trip_id,node_name,vendor_name,rider_payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,rider_fuel_cpo,fm_mm_cpo
0,0,BZPPL00093131,Bangalore,560027,Delivered,Prepaid,35.0,Bangalore,5194.0,3582613.0,STNG-Franchise,STNG-Franchise,ORDER,39.380874,2024-04-01 05:37:24.488000,0.000000,2.943033,11.82
1,1,BZPPL00106754,Hyderabad,500009,Delivered,COD,35.0,Hyderabad,5112.0,3629168.0,DS HYD TRNK,Peel works,FIXED,54.897959,2024-04-03 11:04:37.650000,11.072761,3.888132,19.38
2,2,BZPPL00112314,Hyderabad,500073,Delivered,COD,40.0,Hyderabad,2776.0,3810901.0,DS HYD MHDP,Shiva Shakti,ORDER,36.961131,2024-04-13 05:54:34.224000,1.668828,3.888132,19.38
3,3,BZPPL00117900,Bangalore,560108,RTO Delivered,COD,35.0,Bangalore,0.0,0.0,0,0,0,0.000000,0,0.000000,0.000000,23.64
4,4,BZPPL00118912,Bangalore,560064,Delivered,COD,35.0,Bangalore,4461.0,3674677.0,YLHK-Franchise,YLHK-Franchise,ORDER,39.126529,2024-04-06 05:06:52.942000,0.000000,2.943033,11.82


In [97]:
df_revenue.loc[df_revenue['shipping_city']=='NCR', 'shipping_city'] = 'Delhi'

In [98]:
df_merge = df_march.merge(df_revenue,on='awb',how = 'left')

In [99]:
df_merge.head()

,Unnamed: 0,awb,warehouse_city,drop_pincode,shipment_status,mop,rpo,shipping_city_x,num_rider_id,trip_id,...,vendor_name,rider_payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,rider_fuel_cpo,fm_mm_cpo,shipping_city_y,shipping_pincode,sort_code
0,0,BZPPL00093131,Bangalore,560027,Delivered,Prepaid,35.0,Bangalore,5194.0,3582613.0,...,STNG-Franchise,ORDER,39.380874,2024-04-01 05:37:24.488000,0.000000,2.943033,11.82,Bangalore,560027.0,BLR/STNG/STNG-09
1,1,BZPPL00106754,Hyderabad,500009,Delivered,COD,35.0,Hyderabad,5112.0,3629168.0,...,Peel works,FIXED,54.897959,2024-04-03 11:04:37.650000,11.072761,3.888132,19.38,Hyderabad,500009.0,HYD/TRNK/BOPL-07
2,2,BZPPL00112314,Hyderabad,500073,Delivered,COD,40.0,Hyderabad,2776.0,3810901.0,...,Shiva Shakti,ORDER,36.961131,2024-04-13 05:54:34.224000,1.668828,3.888132,19.38,Hyderabad,500073.0,HYD/MHDP/JUBH-11
3,3,BZPPL00117900,Bangalore,560108,RTO Delivered,COD,35.0,Bangalore,0.0,0.0,...,0,0,0.000000,0,0.000000,0.000000,23.64,Bangalore,560108.0,BLR/UTTR/UTTR-12
4,4,BZPPL00118912,Bangalore,560064,Delivered,COD,35.0,Bangalore,4461.0,3674677.0,...,YLHK-Franchise,ORDER,39.126529,2024-04-06 05:06:52.942000,0.000000,2.943033,11.82,Bangalore,560064.0,BLR/YLHK/YLHK-05


In [100]:
df_merge[df_merge['shipping_city_y'].isna()]

,Unnamed: 0,awb,warehouse_city,drop_pincode,shipment_status,mop,rpo,shipping_city_x,num_rider_id,trip_id,...,vendor_name,rider_payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,rider_fuel_cpo,fm_mm_cpo,shipping_city_y,shipping_pincode,sort_code
67,67,BZPPL00196782,Hyderabad,500035,RTO Delivered,COD,35.0,Hyderabad,0.0,0.0,...,0,0,0.000000,0,0.000000,0.000000,38.76,None,500035.0,HYD/BRBN/BRBN-10
417,417,BZPPL00209655,Hyderabad,500092,RTO Delivered,COD,35.0,Hyderabad,0.0,0.0,...,0,0,0.000000,0,0.000000,0.000000,38.76,None,500092.0,HYD/MHDP/GLKD-14
518,518,BZPPL00222309,Bangalore,500086,Delivered,Prepaid,40.0,Bangalore,3347.0,3692765.0,...,Shiva Shakti,ORDER,31.307858,2024-04-07 05:06:25.497000,1.413578,3.888132,11.82,None,500086.0,HYD/SRNG/KPHB-04
618,618,BZPPL00225359,Hyderabad,500039,RTO Delivered,Prepaid,35.0,Hyderabad,0.0,0.0,...,0,0,0.000000,0,0.000000,0.000000,38.76,None,500039.0,HYD/BRBN/BRBN-10
718,718,BZPPL00227316,Hyderabad,500092,RTO Delivered,Prepaid,35.0,Hyderabad,0.0,0.0,...,0,0,0.000000,0,0.000000,0.000000,38.76,None,500092.0,HYD/MHDP/GLKD-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480171,480171,GS2112665493,Bangalore,500088,RTO Delivered,COD,65.0,Bangalore,0.0,0.0,...,0,0,0.000000,0,0.000000,0.000000,23.64,None,500088.0,HYD/TRNK/KVGD-24
482235,482235,GS2118126295,Delhi,122102,RTO Delivered,Prepaid,446.0,Delhi,0.0,0.0,...,0,0,0.000000,0,0.000000,0.000000,17.64,None,122102.0,BLR/JPNR/JPNR-07
485225,485225,GS2125705088,Bangalore,500039,RTO Delivered,COD,65.0,Bangalore,0.0,0.0,...,0,0,0.000000,0,0.000000,0.000000,23.64,None,500039.0,HYD/MHDP/BNJR-22
486732,486732,GS2129509802,Bangalore,650010,Delivered,Prepaid,54.0,Bangalore,2228.0,3648678.0,...,Hindusree,FIXED,37.159468,2024-04-04 11:27:41.043000,1.540788,2.943033,11.82,None,650010.0,BLR/HBBL/RJNR-07


In [101]:
df_merge.isna().sum()

Unnamed: 0                  0
awb                         0
warehouse_city              0
drop_pincode                0
shipment_status             0
mop                         0
rpo                         0
shipping_city_x             0
num_rider_id                0
trip_id                     0
node_name                   0
vendor_name                 0
rider_payout_structure      0
rider_pay_cpo               0
dispatch_time               0
vendor_cpo                  0
rider_fuel_cpo              0
fm_mm_cpo                   0
shipping_city_y           184
shipping_pincode            0
sort_code                   0
dtype: int64

In [102]:
df_merge.loc[df_merge['shipping_city_y'].isna(), 'shipping_city_y'] = df_merge['shipping_city_x']

In [103]:
df_merge = df_merge.drop(columns=['shipping_city_x']).rename(columns={'shipping_city_y':'shipping_city'})

In [104]:
vendor_margins=df_merge.groupby(['shipping_city','vendor_name','num_rider_id']).aggregate({
    'awb': 'count',
    'rider_pay_cpo': 'sum',
    'vendor_cpo': 'sum',
    'rider_fuel_cpo': 'sum',
    'fm_mm_cpo': 'sum',
    'rpo':'sum'
}).reset_index()
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay_cpo,vendor_cpo,rider_fuel_cpo,fm_mm_cpo,rpo
0,Bangalore,0,0.0,8256,0.0,0.0,0.000000,194335.68,643798.0
1,Bangalore,3WH,4673.0,560,21960.0,0.0,1648.098563,6343.20,74230.0
2,Bangalore,3WH,5204.0,625,19230.0,0.0,1839.395717,7153.50,85046.0
3,Bangalore,3WH,5446.0,70,2135.0,0.0,206.012320,779.40,11847.0
4,Bangalore,3WH,5520.0,305,10900.0,0.0,897.625110,3536.10,35885.0


In [105]:
vendor_margins['vendor_cpo'] = vendor_margins['vendor_cpo'].astype(float)
vendor_margins['fm_mm_cpo'] = vendor_margins['fm_mm_cpo'].astype(float)

In [106]:
vendor_margins.rename(columns={'fuel_cpo':'fuel_pay','mm_cpo':'mm_pay','vendor_cpo':'vendor_pay','rider_pay_cpo':'rider_pay'},inplace=True)

vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay,vendor_pay,rider_fuel_cpo,fm_mm_cpo,rpo
0,Bangalore,0,0.0,8256,0.0,0.0,0.000000,194335.68,643798.0
1,Bangalore,3WH,4673.0,560,21960.0,0.0,1648.098563,6343.20,74230.0
2,Bangalore,3WH,5204.0,625,19230.0,0.0,1839.395717,7153.50,85046.0
3,Bangalore,3WH,5446.0,70,2135.0,0.0,206.012320,779.40,11847.0
4,Bangalore,3WH,5520.0,305,10900.0,0.0,897.625110,3536.10,35885.0


In [107]:
vendor_margins.to_csv('vendor_margins_april_corrected.csv')

In [108]:
df_merge.to_csv('margin_april_correcred.csv')